In [1]:
import os
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # split the text?
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma # connect to chromadb?
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

os.environ["OPENAI_API_KEY"]=""
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGHCAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [2]:
def load_docs(source_links):
    docs = []
    for link in source_links:
        # load individual webpages
        loader = WebBaseLoader(
            web_paths=(link,)
        )
        current_doc = loader.load()
        # print(current_doc)
        docs.extend(current_doc)
    
    return docs


In [3]:
#split and embed

def get_vectorstore_for_docs_Chroma(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore

def get_vectorstore_for_docs_FAISS(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore


In [7]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [4]:
source_links = ['https://en.wikipedia.org/wiki/Royal_Rumble_(2024)',
                'https://en.wikipedia.org/wiki/Elimination_Chamber:_Perth',
                'https://en.wikipedia.org/wiki/WrestleMania_XL',
                'https://en.wikipedia.org/wiki/Backlash_France'
                'https://www.allelitewrestling.com/aew-roster',
                'https://en.wikipedia.org/wiki/Money_in_the_Bank_(2024)',
                'https://en.wikipedia.org/wiki/SummerSlam_(2024)'
            ]
docs = load_docs(source_links)



In [5]:
print(len(docs))

6


In [16]:
chroma_vectorstore = get_vectorstore_for_docs_Chroma(docs)
FAISS_vectorstore = get_vectorstore_for_docs_FAISS(docs)

prompt = hub.pull('rlm/rag-prompt')

# "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA."

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA. Be creative with feuds and include dramatic twists for big match finishs. Do not use the same storyline for 2 different feuds."),
#     ("user", "{question}")
# ])
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)
rag_chain_Chroma = (
    {'context': chroma_vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_FAISS = (
    {'context': FAISS_vectorstore.as_retriever() | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
print(rag_chain_Chroma.invoke('Who won the 2024 men\'s royal rumble match?'))
print(rag_chain_FAISS.invoke('Who won the 2024 men\'s royal rumble match?'))

I don't know.
Cody Rhodes won the 2024 men's Royal Rumble match, becoming the fourth person to win back-to-back Rumbles. He challenged Roman Reigns for SmackDown's Undisputed WWE Universal Championship at WrestleMania XL Night 2.


In [18]:
print(rag_chain_Chroma.invoke('Who is Cody Rhodes likely to face at wrestlemania 40?'))
print(rag_chain_FAISS.invoke('Who is Cody Rhodes likely to face at wrestlemania 40?'))

Cody Rhodes is likely to face either Roman Reigns or Seth Rollins at WrestleMania 40 for the Undisputed WWE Universal Championship. Rhodes expressed interest in challenging Reigns for the title, but Rollins made a case for himself as a more deserving opponent. Rhodes has not made a final decision on who he will face at WrestleMania.
Cody Rhodes is likely to face Roman Reigns at WrestleMania 40 for the Undisputed WWE Universal Championship. Rhodes initially considered challenging Seth Rollins but ultimately chose to go after Reigns after fan backlash and encouragement. The main event of WrestleMania XL Night 2 will feature Reigns defending his title against Rhodes.


In [20]:
print(rag_chain_Chroma.invoke('Give me the list of matches for Wrestlemania 40'))
print(rag_chain_FAISS.invoke('Give me the list of matches for Wrestlemania XL'))

I don't know.
I don't know.


In [11]:
rag_chain.invoke('Give me the list of matches for Wrestlemania 40')

"Here is a potential match card for WrestleMania 40:\n\n1. **WWE Universal Championship Match:**\n   Roman Reigns (c) vs. Brock Lesnar\n\n2. **WWE Championship Match:**\n   Seth Rollins (c) vs. AJ Styles\n\n3. **Dream Match - Cross-Promotion:**\n   Kenny Omega (AEW) vs. Finn Balor\n\n4. **SmackDown Women's Championship Match:**\n   Charlotte Flair (c) vs. Ronda Rousey\n\n5. **Intercontinental Championship Match:**\n   Johnny Gargano (c) vs. Adam Cole\n\n6. **Grudge Match:**\n   Becky Lynch vs. Sasha Banks\n\n7. **Tag Team Championship Match:**\n   The Usos (c) vs. The New Day\n\n8. **Special Attraction Match:**\n   The Rock vs. Triple H\n\n9. **RAW Women's Championship Match:**\n   Bianca Belair (c) vs. Bayley\n\n10. **Andre the Giant Memorial Battle Royal**\n\n11. **Cruiserweight Championship Match:**\n    Ricochet (c) vs. Pete Dunne\n\n12. **Legends Match:**\n    Shawn Michaels vs. Edge\n\nThis match card features a mix of championship matches, dream matches, cross-promotion opportun

In [21]:
rag_chain_FAISS.invoke("What is your job?")

'I am an assistant for question-answering tasks.'

In [22]:
rag_chain_FAISS.invoke('Give me the list of matches for Wrestlemania 2024')

'The matches for WrestleMania 2024 have not been officially announced yet. Cody Rhodes and Bayley won the Royal Rumble Matches to earn the right to headline WrestleMania. The event will take place as a two-night event on April 6 and 7, 2024, at Lincoln Financial Field in Philadelphia, Pennsylvania.'

In [23]:
template = """You are the creative head for WWE and are in-charge of booking matches, scripting storylines and promoting premium live events. You must stick to WWE's active roster to involve in feuds. On, ocassion, you can include a cross-over match with rival promotions like AEW, NJPW, TNA. In addition, you will answer questions based on the provided context and past history. Do not answer any questions involving AEW.
Use the following context to augment your existing knowledge in the field: {context}

Question: {question}
"""

new_prompt = ChatPromptTemplate.from_template(template)

booker_rag_chain = (
    {
        'context': FAISS_vectorstore.as_retriever() | format_docs,
        'question': RunnablePassthrough()
    }
    | new_prompt
    | llm
    | StrOutputParser()
)

In [24]:
booker_rag_chain.invoke("Introduce yourself")

"Hello, I am the creative head for WWE, responsible for booking matches, scripting storylines, and promoting premium live events. I work with WWE's active roster to create exciting feuds and matches for our fans to enjoy."

In [25]:
booker_rag_chain.invoke('Who owns AEW?')

'AEW is owned by Tony Khan.'

In [26]:
booker_rag_chain.invoke('Who won the latest royal rumble')

"Cody Rhodes won the men's Royal Rumble match for the second consecutive year."

In [27]:
booker_rag_chain.invoke('Who is the other royal rumble winner?')

"Bayley is the winner of the 2024 women's Royal Rumble match."

In [28]:
booker_rag_chain.invoke('What are the confirmed matches for Wrestlemania?')

"As the creative head for WWE, the confirmed matches for WrestleMania XL in 2024 include Roman Reigns defending the Universal Championship against Cody Rhodes, Becky Lynch defending the Raw Women's Championship against Sasha Banks, and The New Day defending the Raw Tag Team Championships against The Usos."

In [29]:
booker_rag_chain.invoke("Give me the results of the premium live-event that was held in Australia")

"The results of the premium live-event Elimination Chamber: Perth held in Australia on February 24, 2024, are as follows:\n\n1. Roman Reigns defeated Brock Lesnar, Seth Rollins, Bobby Lashley, Kevin Owens, and Finn Balor in the Men's Elimination Chamber match to retain the Universal Championship.\n\n2. Rhea Ripley defeated Bianca Belair, Sasha Banks, Charlotte Flair, Becky Lynch, and Bayley in the Women's Elimination Chamber match to retain the Raw Women's Championship.\n\n3. The Usos (Jimmy and Jey Uso) defeated The New Day (Kofi Kingston and Xavier Woods) to retain the SmackDown Tag Team Championships.\n\n4. AJ Styles defeated Shinsuke Nakamura in a singles match.\n\n5. Edge defeated Randy Orton in a Last Man Standing match.\n\n6. The Miz and John Morrison defeated The Street Profits (Angelo Dawkins and Montez Ford) to win the Raw Tag Team Championships.\n\n7. Sheamus defeated Drew McIntyre in a Falls Count Anywhere match.\n\n8. Asuka defeated Alexa Bliss in a singles match."